In [64]:
import math

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import multiprocessing

import huggingface_hub
import os
from dotenv import load_dotenv

from transformers import AutoConfig, T5Config
from transformers import AutoTokenizer, T5TokenizerFast
from transformers import DataCollatorWithPadding, DataCollatorForSeq2Seq
from transformers import AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Seq2SeqTrainingArguments
from transformers import Trainer, Seq2SeqTrainer
from transformers import pipeline

import datasets
from datasets import load_dataset #, load_from_disk


from mamba_ssm import Mamba
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from einops import rearrange

import tqdm as notebook_tqdm
from tqdm.auto import tqdm



In [65]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device_count = torch.cuda.device_count()
# torch.cuda.empty_cache()
cpu_cores = multiprocessing.cpu_count()
print(device, device_count, f'cpu:{cpu_cores}')

cuda 2 cpu:8


In [66]:
load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

In [67]:
batch, length, dim = 2, 64, 16
x = torch.randn(batch, length, dim).to("cuda")
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
y = model(x)
assert y.shape == x.shape

In [73]:
# model_checkpoint = "state-spaces/mamba-2.8b"
modelHead_checkpoint_mamba = "state-spaces/mamba-130m"
tokenizer_checkpoint_mamba = "EleutherAI/gpt-neox-20b"

tokenizer_mamba = AutoTokenizer.from_pretrained(tokenizer_checkpoint_mamba)
model_mamba = MambaLMHeadModel.from_pretrained(modelHead_checkpoint_mamba, device=device, dtype=torch.float16).to(device)
# config = AutoConfig.from_pretrained("state-spaces/mamba-2.8b")

In [74]:
model_checkpoint_t5 = "t5-small"

tokenizer_t5 = AutoTokenizer.from_pretrained(model_checkpoint_t5)
# config_t5 = AutoConfig.from_pretrained(model_checkpoint_t5, output_hidden_states=True)
# print(config_t5)
model_t5 = AutoModel.from_pretrained(model_checkpoint_t5).to(device)
# print(model_t5.config)

In [70]:
# # !touch .env
# # !touch .gitignore
# # !echo '.env' >> .gitignore
# # !echo 'HUGGINGFACE_TOKEN="hf_ONAr"' >> .env'
# load_dotenv()
# huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
# # print(huggingface_token)
# # huggingface_hub.login(token=huggingface_token, add_to_git_credential=True)